In [1]:
import pandas as pd
import numpy as np
import re
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

In [2]:
# ──────────────── CONFIGURATION ────────────────

x = 5000


# 1) Desired number of accepted/pending/blocked examples for each label:
DESIRED_COUNTS = {
    "Bullying":             {"accepted": x, "pending": x, "blocked": x},
    "Fighting":             {"accepted": x, "pending": x, "blocked": x},
    "Sexting":              {"accepted": x, "pending": x, "blocked": x},
    "Vulgar":               {"accepted": x, "pending": x, "blocked": x},
    "Drugs":                {"accepted": x, "pending": x, "blocked": x},
    "InGame":               {"accepted": x, "pending": x, "blocked": x},
    "Alarm":                {"accepted": x, "pending": x, "blocked": x},
    "Fraud":                {"accepted": x, "pending": x, "blocked": x},
    "Racist":               {"accepted": x, "pending": x, "blocked": x},
    "Religion":             {"accepted": x, "pending": x, "blocked": x},
    "Junk":                 {"accepted": x, "pending": x, "blocked": x},
    "Website":              {"accepted": x, "pending": x, "blocked": x},
    "Grooming":             {"accepted": x, "pending": x, "blocked": x},
    #"PublicThreats":        {"accepted": x, "pending": x, "blocked": x},
  #  "RealName":             {"accepted": x, "pending": x, "blocked": x},
    #"ExtremistRecruitment": {"accepted": x, "pending": x, "blocked": x},
   # "Subversive":           {"accepted": x, "pending": x, "blocked": x},
   # "Sentiment":            {"accepted": x, "pending": x, "blocked": x},
    "Politics":             {"accepted": x, "pending": x, "blocked": x},
    # "Nothing Wrong" only uses 'accepted':
    "Nothing Wrong":        {"accepted": 1.5*x, "pending": 0, "blocked": 0},
}

# 2) Compute global totals dynamically from DESIRED_COUNTS
DESIRED_OVERALL = {"accepted": 0, "pending": 0, "blocked": 0}
for label_counts in DESIRED_COUNTS.values():
    for decision in DESIRED_OVERALL:
        DESIRED_OVERALL[decision] += label_counts.get(decision, 0)

# ──────────────── END CONFIGURATION ────────────────

# Running counts that mirror DESIRED_COUNTS but start at 0
counts = {
    label: {"accepted": 0, "pending": 0, "blocked": 0}
    for label in DESIRED_COUNTS
}

# Global totals to track progress
global_counts = {"accepted": 0, "pending": 0, "blocked": 0}

# To track duplicate texts
seen_texts = set()

print(DESIRED_OVERALL)



{'accepted': 77500.0, 'pending': 70000, 'blocked': 70000}


In [3]:
# ──────────────── STEP 1: Helper to tell us if we should stop ────────────────
def stop_criteria_met():
    """
    Return True if EITHER
      a) ALL per-label targets are reached (for every label, each decision bucket is >= desired),
      OR
      b) ALL global targets are reached (accepted >= DESIRED_OVERALL['accepted'], etc.).
    """
    # a) Check per-label:
    all_labels_full = True
    for label, desired in DESIRED_COUNTS.items():
        for dec in ["accepted", "pending", "blocked"]:
            # If DESIRED_COUNTS[label][dec] is zero, we consider it "already satisfied"
            # as long as our counts[label][dec] >= 0 (which is always true).
            if desired[dec] > 0 and counts[label][dec] < desired[dec]:
                all_labels_full = False
                break
        if not all_labels_full:
            break

    # b) Check global:
    all_global_full = True
    for dec in ["accepted", "pending", "blocked"]:
        if global_counts[dec] < DESIRED_OVERALL[dec]:
            all_global_full = False
            break

    return all_labels_full or all_global_full



In [4]:
# ──────────────── STEP 2: Process one CSV file ────────────────
def process_csv_file(path):
    """
    Reads `path` into a DataFrame, filters for English rows without {{…}} tokens,
    then for each row decides:
      - if all label‐columns are zero → treat as "Nothing Wrong"
      - otherwise find the first non-zero label, map its status → decision,
        check if that label‐decision bucket is still under its DESIRED_COUNTS,
        and if so, add it. Also skip duplicates on `text`.
    Returns a list of row‐dicts to append, and updates the global counts.
    """
    df = pd.read_csv(path)
    # Keep only English:
    df = df[df["user_primary_language"] == "en"]
    # Filter out any row whose text contains {{…}}:
    df = df[~df["text"].astype(str).str.contains(r"\{\{.*?\}\}", regex=True)]

    # Only keep these label columns (plus text & language):
    label_cols = [
        "Bullying", "Fighting", "Sexting", "Vulgar", "Drugs", "InGame", "Alarm", "Fraud",
        "Racist", "Religion", "Junk", "Website", "Grooming", "Politics"
    ]

#"PublicThreats""Sentiment""ExtremistRecruitment"

    df[label_cols] = df[label_cols].fillna(0)

    rows_to_add = []

    for _, row in df.iterrows():
        if stop_criteria_met():
            # If either per-label OR global targets are satisfied, stop immediately.
            break

        text = row["text"]
        # 1) Dedupe on text:
        if text in seen_texts:
            continue

        # 2) Check if all label_cols are zero → "Nothing Wrong"
        is_all_zero = True
        for lc in label_cols:
            if int(row[lc]) != 0:
                is_all_zero = False
                break

        # If "Nothing Wrong":
        if is_all_zero:
            label = "Nothing Wrong"
            decision = "accepted"  # always accepted for "Nothing Wrong"
            # Check if we still need more of this:
            if counts[label][decision] < DESIRED_COUNTS[label][decision]:
                # Add one
                counts[label][decision] += 1
                global_counts[decision] += 1
                seen_texts.add(text)
                rows_to_add.append({
                    "Text": text,
                    "Label": label,
                    "Status": 0,
                    "Decision": decision
                })
            # either way, skip any further labels 
            continue

        # Otherwise, find the first non-zero label in label_cols:
        for lc in label_cols:
            status = int(row[lc])
            if status == 0:
                continue

            # Map status → decision:
            #   0,1,2  → accepted
            #   3,4    → pending
            #   5,6,7  → blocked
            if status in [0, 1, 2]:
                decision = "accepted"
            elif status in [3, 4]:  
                decision = "pending"
            elif status in [5, 6, 7]:
                decision = "blocked"
            else:
                # any other status code we skip:
                continue

            label = lc  # e.g. "Bullying", "Fighting", etc.
            # If we still need more of this label/decision:
            if counts[label][decision] < DESIRED_COUNTS[label][decision]:
                counts[label][decision] += 1
                global_counts[decision] += 1
                seen_texts.add(text)
                rows_to_add.append({
                    "Text": text,
                    "Label": label,
                    "Status": status,
                    "Decision": decision
                })
            # Whether we added or not, stop checking other labels for this text:
            break

    return rows_to_add

In [5]:
# ──────────────── STEP 3: Summary‐report generator ────────────────
def generate_summary_report(all_rows_df):
    """
    all_rows_df has columns: Text, Label, Status, Decision.
    We now show:
      1) For each label: #accepted, #pending, #blocked
      2) Overall accepted/pending/blocked totals
    (Total column has been removed as requested)
    """
    summary_data = []

    # A) Overall decision summary:
    overall = all_rows_df["Decision"].value_counts().to_dict()
    summary_data.append(["Overall accepted", overall.get("accepted", 0)])
    summary_data.append(["Overall pending", overall.get("pending", 0)])
    summary_data.append(["Overall blocked", overall.get("blocked", 0)])
    summary_data.append([])  # Empty row for spacing

    # B) Per-label breakdown (including "Nothing Wrong"):
    summary_data.append(["Label", "Accepted", "Pending", "Blocked"])  # Removed "Total"
    labels = list(DESIRED_COUNTS.keys())
    
    for label in labels:
        a = all_rows_df[(all_rows_df["Label"] == label) & (all_rows_df["Decision"] == "accepted")].shape[0]
        p = all_rows_df[(all_rows_df["Label"] == label) & (all_rows_df["Decision"] == "pending")].shape[0]
        b = all_rows_df[(all_rows_df["Label"] == label) & (all_rows_df["Decision"] == "blocked")].shape[0]
        summary_data.append([label, a, p, b])  # Removed total calculation

    # Save to CSV:
    summary_df = pd.DataFrame(summary_data)
    summary_df.to_csv("summary_report_csv.csv", index=False, header=False, encoding="utf-8-sig")

    # Also print to console:
    print("\n📊 Summary Report (Totals Removed):\n")
    print("Overall accepted :", overall.get("accepted", 0))
    print("Overall pending  :", overall.get("pending", 0))
    print("Overall blocked  :", overall.get("blocked", 0))
    print()
    print(f"{'Label':25s}{'Acc':>8s}{'Pen':>8s}{'Blk':>8s}")  # Removed 'Total' from header
    for row in summary_data[5:]:  # skip first five rows (overall counts and empty row and header)
        label, a, p, b = row  # Removed tot from unpacking
        print(f"{label:25s}{a:8d}{p:8d}{b:8d}")  # Removed tot from print

In [6]:
# ──────────────── STEP 4: Process entire folder ────────────────
def file_clean_summary_for_folder(folder_path):
    all_rows = []

    for fname in os.listdir(folder_path):
        if not fname.lower().endswith(".csv"):
            continue
        fullpath = os.path.join(folder_path, fname)
        print(f"🔄 Processing file: {fname}…")

        new_rows = process_csv_file(fullpath)
        all_rows.extend(new_rows)

        if stop_criteria_met():
            print("✅ Stop criteria met → ending early.")
            break

    # Convert to DataFrame (without the TotalForThatLabel column)
    final_df = pd.DataFrame(all_rows, columns=["Text", "Label", "Status", "Decision"])
    
    # Save the final CSV:
    final_df.to_csv("final_csv.csv", index=False, encoding="utf-8-sig")
    print("✅ Saved `final_csv.csv` with deduplication, per-label/per-decision caps applied.")
    
    # Now produce the summary CSV:
    generate_summary_report(final_df)
    

In [7]:
# ──────────────── STEP 5: Run on your folder ────────────────
folder_path = r"U:\N\csv"
file_clean_summary_for_folder(folder_path)

🔄 Processing file: 1.csv…
🔄 Processing file: 1724275399_chat_logs_to_csv_0_fe6260b29bf546d984368cebd17431e0.csv…
🔄 Processing file: 1724275399_chat_logs_to_csv_1_fe6260b29bf546d984368cebd17431e0.csv…
🔄 Processing file: 1727392003_chat_logs_to_csv_0_cc1dcac030ce49328aa3da017ba0a963.csv…
🔄 Processing file: 1727392003_chat_logs_to_csv_1_cc1dcac030ce49328aa3da017ba0a963.csv…
🔄 Processing file: 1727392003_chat_logs_to_csv_2_cc1dcac030ce49328aa3da017ba0a963.csv…
🔄 Processing file: 1727392003_chat_logs_to_csv_3_cc1dcac030ce49328aa3da017ba0a963.csv…
🔄 Processing file: 1727392003_chat_logs_to_csv_4_cc1dcac030ce49328aa3da017ba0a963.csv…
🔄 Processing file: 1748861002_chat_logs_to_csv_0_2bc0d3c6185d4df8af4c0abae04e85cd.csv…
🔄 Processing file: 1748861002_chat_logs_to_csv_1_2bc0d3c6185d4df8af4c0abae04e85cd.csv…
🔄 Processing file: 1748861002_chat_logs_to_csv_2_2bc0d3c6185d4df8af4c0abae04e85cd.csv…
🔄 Processing file: 1748861002_chat_logs_to_csv_3_2bc0d3c6185d4df8af4c0abae04e85cd.csv…
🔄 Processing file

In [8]:
import csv
import json
import sys
import random

# Fixed list of moderation categories in desired order
CATEGORIES = [
    "Junk", "Vulgar", "Nothing Wrong", "Bullying",
    "Fighting", "Sexting", "Drug References", "In-Game Content",
    "Fraud", "Racism", "Sentiment", "Grooming Attempts",
    "RealName", "Website", "Alarm", "Politics", "Religion", "Subversive"
]

# System instruction with added severity interpretation
SYSTEM_TEXT = (
    "You are analyzing a message from the Animal Jam chat system to determine "
    "if it contains any inappropriate or harmful content that may violate community guidelines.\n\n"
    "For each relevant category (Bullying, Fighting, Sexting, Vulgar Language, Drug References, "
    "In-Game Content, Fraud, Racism, Sentiment, Grooming Attempts), assign a severity score 0–7. "
    "Output ONLY the JSON object with all categories (zeros included) in this fixed key order.\n\n"
    "Severity interpretation:\n"
    "  - 0 to 2: Safe for kids\n"
    "  - 3 to 4: Might be bad but okish\n"
    "  - 5 to 7: Bad and should not be read by kids"
)

input_file = r'U:\N\final_csv.csv'
train_output_file = 'gemini_ft_train.jsonl'
val_output_file = 'gemini_ft_val.jsonl'
VAL_RATIO = 0.1  # 10% of data will go to validation

def build_record(text, label, severity):
    scores = {cat: 0 for cat in CATEGORIES}
    if label in scores:
        scores[label] = severity

    return {
        "systemInstruction": {
            "role": "system",
            "parts": [{"text": SYSTEM_TEXT}]
        },
        "contents": [
            {"role": "user", "parts": [{"text": text}]},
            {"role": "model", "parts": [{"text": json.dumps(scores, separators=(',', ':'))}]}
        ]
    }

def convert_and_split_csv():
    with open(input_file, newline='', encoding='utf-8') as csvf:
        reader = list(csv.DictReader(csvf))
        random.shuffle(reader)

        split_idx = int(len(reader) * (1 - VAL_RATIO))
        train_rows = reader[:split_idx]
        val_rows = reader[split_idx:]

        train_count = 0
        val_count = 0

        with open(train_output_file, 'w', encoding='utf-8') as train_outf, \
             open(val_output_file, 'w', encoding='utf-8') as val_outf:

            for row in train_rows:
                text = row.get('\ufeffText', row.get('Text', '')).strip()
                label = row.get('Label', '').strip()
                raw = row.get('Status', '0').strip()
                try:
                    severity = int(float(raw))
                except ValueError:
                    severity = 0

                record = build_record(text, label, severity)
                train_outf.write(json.dumps(record, ensure_ascii=False) + "\n")
                train_count += 1

            for row in val_rows:
                text = row.get('\ufeffText', row.get('Text', '')).strip()
                label = row.get('Label', '').strip()
                raw = row.get('Status', '0').strip()
                try:
                    severity = int(float(raw))
                except ValueError:
                    severity = 0

                record = build_record(text, label, severity)
                val_outf.write(json.dumps(record, ensure_ascii=False) + "\n")
                val_count += 1

    print(f"Training records written: {train_count}")
    print(f"Validation records written: {val_count}")


def validate_first_record(path):
    with open(path, encoding='utf-8') as f:
        line = f.readline()
    try:
        rec = json.loads(line)
    except json.JSONDecodeError as e:
        print(f"JSON parse error in {path}: {e}")
        sys.exit(1)
    if 'systemInstruction' not in rec or 'contents' not in rec:
        print(f"Format error in {path}: Missing 'systemInstruction' or 'contents'")
        sys.exit(1)
    if not isinstance(rec['contents'], list) or len(rec['contents']) != 2:
        print(f"Format error in {path}: 'contents' must be a list of two entries (user, model)")
        sys.exit(1)
    print(f"Validation passed: {path} is Gemini-compatible JSONL.")

if __name__ == '__main__':
    convert_and_split_csv()
    validate_first_record(train_output_file)
    validate_first_record(val_output_file)


Training records written: 153325
Validation records written: 17037
Validation passed: gemini_ft_train.jsonl is Gemini-compatible JSONL.
Validation passed: gemini_ft_val.jsonl is Gemini-compatible JSONL.


In [9]:
# import csv
# import json
# import sys

# # Fixed list of moderation categories in desired order
# CATEGORIES = [
#     "Junk", "Vulgar", "Nothing Wrong", "Bullying",
#     "Fighting", "Sexting", "Drug References", "In-Game Content",
#     "Fraud", "Racism", "Sentiment", "Grooming Attempts",
#     "RealName", "Website", "Alarm", "Politics", "Religion", "Subversive"
# ]

# # System instruction with added severity interpretation
# SYSTEM_TEXT = (
#     "You are analyzing a message from the Animal Jam chat system to determine "
#     "if it contains any inappropriate or harmful content that may violate community guidelines.\n\n"
#     "For each relevant category (Bullying, Fighting, Sexting, Vulgar Language, Drug References, "
#     "In-Game Content, Fraud, Racism, Sentiment, Grooming Attempts), assign a severity score 0–7. "
#     "Output ONLY the JSON object with all categories (zeros included) in this fixed key order.\n\n"
#     "Severity interpretation:\n"
#     "  - 0 to 2: Safe for kids\n"
#     "  - 3 to 4: Might be bad but okish\n"
#     "  - 5 to 7: Bad and should not be read by kids"
# )

# input_file = r'U:\N\final_csv.csv'
# output_file = 'gemini_ft_dataset.jsonl'


# def convert_csv_to_gemini_jsonl():
#     with open(input_file, newline='', encoding='utf-8') as csvf, \
#          open(output_file, 'w', encoding='utf-8') as outf:

#         reader = csv.DictReader(csvf)
#         for row in reader:
#             # Extract text and label
#             text = row.get('\ufeffText', row.get('Text', '')).strip()
#             label = row.get('Label', '').strip()
#             raw = row.get('Status', '0').strip()
#             try:
#                 severity = int(float(raw))
#             except ValueError:
#                 severity = 0

#             # Populate all categories with zero, then set the one label
#             scores = {cat: 0 for cat in CATEGORIES}
#             if label in scores:
#                 scores[label] = severity

#             # Build the Gemini-style record
#             record = {
#                 "systemInstruction": {
#                     "role": "system",
#                     "parts": [{"text": SYSTEM_TEXT}]
#                 },
#                 "contents": [
#                     {
#                         "role": "user",
#                         "parts": [{"text": text}]
#                     },
#                     {
#                         "role": "model",
#                         "parts": [{"text": json.dumps(scores, separators=(',', ':'))}]
#                     }
#                 ]
#             }
#             outf.write(json.dumps(record, ensure_ascii=False) + "\n")


# def validate_first_record():
#     # Quick check of structure
#     with open(output_file, encoding='utf-8') as f:
#         line = f.readline()
#     try:
#         rec = json.loads(line)
#     except json.JSONDecodeError as e:
#         print(f"JSON parse error: {e}")
#         sys.exit(1)
#     # Check required fields
#     if 'systemInstruction' not in rec or 'contents' not in rec:
#         print("Format error: Missing 'systemInstruction' or 'contents' field")
#         sys.exit(1)
#     if not isinstance(rec['contents'], list) or len(rec['contents']) != 2:
#         print("Format error: 'contents' must be a list of two entries (user, model)")
#         sys.exit(1)
#     print("Validation passed: Dataset is Gemini-compatible JSONL.")


# if __name__ == '__main__':
#     convert_csv_to_gemini_jsonl()
#     validate_first_record()


In [10]:
# import csv
# import json
# from collections import OrderedDict

# # Your fixed list of categories, in the exact order you want them in the output JSON
# CATEGORIES = [
#     "Junk", "Vulgar", "Nothing Wrong", "Bullying",
#     "Fighting", "Sexting", "Drug References", "In-Game Content",
#     "Fraud", "Racism", "Sentiment", "Grooming Attempts",
#     "RealName", "Website", "Alarm", "Politics", "Religion", "Subversive"
# ]

# # System instruction with added severity interpretation
# SYSTEM_TEXT = (
#     "You are analyzing a message from the Animal Jam chat system to determine "
#     "if it contains any inappropriate or harmful content that may violate community guidelines.\n\n"
#     "For each relevant category (Bullying, Fighting, Sexting, Vulgar Language, Drug References, "
#     "In-Game Content, Fraud, Racism, Sentiment, Grooming Attempts), assign a severity score 0–7. "
#     "Output ONLY the JSON object with all categories (zeros included) in this fixed key order.\n\n"
#     "Severity interpretation:\n"
#     "  - 0 to 2: Safe for kids\n"
#     "  - 3 to 4: Might be bad but okish\n"
#     "  - 5 to 7: Bad and should not be read by kids"
# )

# input_file = r'U:\N\final_csv.csv'
# output_file = 'ft_input.jsonl'

# with open(input_file, newline='', encoding='utf-8') as csvfile, \
#      open(output_file, 'w', encoding='utf-8') as jsonlfile:

#     reader = csv.DictReader(csvfile)
#     for row in reader:
#         # grab the text field (handle BOM if present)
#         text = row.get('\ufeffText', row.get('Text', '')).strip()
#         label = row.get('Label', '').strip()
#         raw_status = row.get('Status', '0').strip()
#         try:
#             severity = int(float(raw_status))
#         except ValueError:
#             severity = 0

#         # Build a full dict of all categories, zeros by default
#         scores = OrderedDict((cat, 0) for cat in CATEGORIES)
#         # only overwrite the one category this row labels
#         if label in scores:
#             scores[label] = severity

#         # Serialize the JSON object with no additional whitespace
#         completion_str = json.dumps(scores, separators=(',', ':'))

#         # Build the prompt: system + user + assistant marker
#         prompt = (
#             f"<|system|> {SYSTEM_TEXT}\n"
#             f"<|user|> {text}\n"
#             f"<|assistant|>"
#         )

#         # Vertex AI wants a leading space before the completion, and a trailing newline
#         record = {
#             "prompt": prompt,
#             "completion": " " + completion_str + "\n"
#         }

#         jsonlfile.write(json.dumps(record) + "\n")


In [11]:
# import pandas as pd

# # Load your CSV (adjust path as needed)
# df = pd.read_csv("download.csv")

# # Replace "locked" with "blocked"
# df['Decision'] = df['Decision'].replace('locked', 'blocked')

# # Save to desired drive and folder
# df.to_csv(r"U:\N\updated_decision_column.csv", index=False)
# print("File saved to U:\\N")


In [12]:
# import pandas as pd

# # Paths to your input files
# file1 = 'final_csv.csv'
# file2 = 'updated_decision_column.csv'

# # Read them into DataFrames
# df1 = pd.read_csv(file1)
# df2 = pd.read_csv(file2)

# # Concatenate vertically (one below the other)
# combined_df = pd.concat([df1, df2], ignore_index=True)

# # Optionally: drop duplicate rows, if needed
# # combined_df = combined_df.drop_duplicates()

# # Write out to a new CSV
# output_file = 'combined_output.csv'
# combined_df.to_csv(output_file, index=False)

# print(f"Combined CSV saved to: {output_file}")

